In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install --upgrade tqdm
from tqdm import tqdm

In [3]:
!pip install --upgrade pandas
!pip3 install pickle5
import pickle5 as pickle

     |████████████████████████████████| 256 kB 4.2 MB/s 


In [4]:
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
!pip install catboost
!pip install category_encoders

     |████████████████████████████████| 76.1 MB 1.1 MB/s 
     |████████████████████████████████| 82 kB 388 kB/s 


In [6]:
import warnings
warnings.filterwarnings(action='ignore') # default
pd.set_option('mode.chained_assignment', None)

In [7]:
from catboost import CatBoostRegressor
RANDOM_STATE = 42
learning_rate = 0.05
n_estimators = 5000

model = CatBoostRegressor(
    iterations=None, learning_rate=learning_rate, depth=None,
    loss_function='RMSE', verbose=False,
    eval_metric='RMSE', max_depth=None, n_estimators=n_estimators, task_type='GPU',
    boosting_type=None, random_state=RANDOM_STATE
    )

In [8]:
showtime = pd.read_parquet('/content/drive/MyDrive/현대홈쇼핑/Data/host_weather_data.parquet')
showtime

,방송편성번호,상품구분,방송일시,PGM코드,쇼호스트명,방송시작시간,방송종료시간,방송길이,날씨,기온,강수량,강설량,강수확률
0,201701013001,1,20170101,205650,"김민성, 김은화",2017-01-01 06:00:00,2017-01-01 06:50:00,50,구름 조금,1.0,0.0,0.0,10
1,201701013002,1,20170101,204693,"김준호, 신진아",2017-01-01 06:50:00,2017-01-01 07:45:00,55,구름 조금,1.0,0.0,0.0,10
2,201701013003,1,20170101,205821,"유경미, 현세환",2017-01-01 07:45:00,2017-01-01 08:45:00,60,구름 조금,1.0,0.0,0.0,10
3,201701013004,1,20170101,204129,"김동은, 이여주",2017-01-01 08:45:00,2017-01-01 11:15:00,150,구름 많음,5.0,0.0,0.0,20
4,201701013005,1,20170101,204946,심용수,2017-01-01 11:15:00,2017-01-01 12:30:00,75,구름 많음,5.0,0.0,0.0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83559,202112315024,0,20211231,223131,심용수,2021-12-31 18:40:00,2021-12-31 19:40:00,60,맑음,-8.0,0.0,0.0,0
83560,202112315025,0,20211231,222790,"박민아, 이호준",2021-12-31 19:40:00,2021-12-31 20:40:00,60,맑음,-8.0,0.0,0.0,0
83561,202112315026,0,20211231,223006,장영재,2021-12-31 20:40:00,2021-12-31 21:40:00,60,맑음,-8.0,0.0,0.0,0
83562,202112315027,0,20211231,221984,"김덕윤, 안소영",2021-12-31 21:40:00,2021-12-31 23:00:00,80,맑음,-8.0,0.0,0.0,0


In [9]:
with open('/content/drive/Shareddrives/현대홈쇼핑/ProcessedData/join용 데이터(최신)/0215_2158_holidayonly.pkl', 'rb') as f:
  holiday = pickle.load(f)[['date','주말제외공휴일']]
holiday['date'] = holiday['date'].astype('str').str.replace('-','').astype('int')
holiday = holiday.rename(columns={'date':'방송일시'})
holiday

,방송일시,주말제외공휴일
0,20200101,1
1,20200102,0
2,20200103,0
3,20200104,0
4,20200105,0
...,...,...
726,20211227,0
727,20211228,0
728,20211229,0
729,20211230,0


In [10]:
# 데이터 불러오기
with open('/content/drive/Shareddrives/현대홈쇼핑/ProcessedData/0211_1353_skill2pgm_1차완성_ver3.pkl', 'rb') as f:
  df = pickle.load(f)
df = df.rename(columns={'comb_상품중분류명':'상품중분류명','comb_브랜드명':'브랜드명'})
df = df[['방송편성번호','인입콜',# 'approx_in_show_call', 'comb_판매상품명',
         '상품중분류명', # 'comb_상품대분류명', 'comb_상품소분류명','comb_상품세분류명',
         '브랜드명','상품판매가','한정표현구분','한정표현_num']]

In [11]:
df_test = pd.read_parquet('/content/drive/MyDrive/현대홈쇼핑/Modeling/21년 데이터/1월데이터.parquet')
df_test['상품구분'] = df_test['상품구분'].replace({'DATA':0,'LIVE':1})
df_test = df_test.drop('한정표현구분',axis=1).rename(columns={'한정표현_list':'한정표현구분','주말/공휴일':'주말제외공휴일','duration':'방송길이',})

df_test = df_test[['방송편성번호','PGM코드','상품구분','방송일시','쇼호스트명','방송시작시간','방송종료시간','방송길이','날씨','기온','인입콜',
                   '상품중분류명','브랜드명','상품판매가','한정표현구분','한정표현_num','주말제외공휴일']]

df_test['쇼호스트명'] = df_test['쇼호스트명'].apply(lambda x: str(x)[2:-2].replace("'",'').replace(' ',', '))
df_test['한정표현구분'] = df_test['한정표현구분'].apply(lambda x: ['없음'] if list(x)==[None] else list(x))

In [12]:
df = pd.merge(showtime,df,on='방송편성번호')
df = pd.merge(df,holiday,on='방송일시')

In [13]:
df['YEAR'] = df['방송시작시간'].apply(lambda x: x.year)
df['MONTH'] = df['방송시작시간'].apply(lambda x: x.month)
df['DAY'] = df['방송시작시간'].apply(lambda x: x.day)
df['WEEKDAY'] = df['방송시작시간'].apply(lambda x: x.weekday())
df['HOUR_START'] = df['방송시작시간'].apply(lambda x: x.hour)
df['HOUR_END'] = df['방송종료시간'].apply(lambda x: x.hour)

df['MONTH_SIN'] = np.sin(2*np.pi*df['MONTH']/12)
df['MONTH_COS'] = np.cos(2*np.pi*df['MONTH']/12)
df['WEEKDAY_SIN'] = np.sin(2*np.pi*df['WEEKDAY']/7)
df['WEEKDAY_COS'] = np.cos(2*np.pi*df['WEEKDAY']/7)
df['HOUR_START_SIN'] = np.sin(2*np.pi*df['HOUR_START']/24)
df['HOUR_START_COS'] = np.cos(2*np.pi*df['HOUR_START']/24)
df['HOUR_END_SIN'] = np.sin(2*np.pi*df['HOUR_END']/24)
df['HOUR_END_COS'] = np.cos(2*np.pi*df['HOUR_END']/24)

In [14]:
df_test['YEAR'] = df_test['방송시작시간'].apply(lambda x: x.year)
df_test['MONTH'] = df_test['방송시작시간'].apply(lambda x: x.month)
df_test['DAY'] = df_test['방송시작시간'].apply(lambda x: x.day)
df_test['WEEKDAY'] = df_test['방송시작시간'].apply(lambda x: x.weekday())
df_test['HOUR_START'] = df_test['방송시작시간'].apply(lambda x: x.hour)
df_test['HOUR_END'] = df_test['방송종료시간'].apply(lambda x: x.hour)

df_test['MONTH_SIN'] = np.sin(2*np.pi*df_test['MONTH']/12)
df_test['MONTH_COS'] = np.cos(2*np.pi*df_test['MONTH']/12)
df_test['WEEKDAY_SIN'] = np.sin(2*np.pi*df_test['WEEKDAY']/7)
df_test['WEEKDAY_COS'] = np.cos(2*np.pi*df_test['WEEKDAY']/7)
df_test['HOUR_START_SIN'] = np.sin(2*np.pi*df_test['HOUR_START']/24)
df_test['HOUR_START_COS'] = np.cos(2*np.pi*df_test['HOUR_START']/24)
df_test['HOUR_END_SIN'] = np.sin(2*np.pi*df_test['HOUR_END']/24)
df_test['HOUR_END_COS'] = np.cos(2*np.pi*df_test['HOUR_END']/24)

In [15]:
# df['approx_in_show_call'] = round(df['approx_in_show_call']).astype('int')

df['쇼호스트_num'] = df['쇼호스트명'].apply(lambda x: 0 if x=='없음' else len(x.split(', ')))
df['상품중분류_num'] = df['상품중분류명'].apply(lambda x: len(set(x)))
df['브랜드_num'] = df['브랜드명'].apply(lambda x: len(set(x)))
df['판매상품개수'] = df['상품중분류명'].apply(lambda x: len(x))

df['상품판매가_min'] = df['상품판매가'].apply(lambda x: min(x))
df['상품판매가_max'] = df['상품판매가'].apply(lambda x: max(x))
df['상품판매가_mean'] = df['상품판매가'].apply(lambda x: np.mean(x))

In [16]:
# df_test['approx_in_show_call'] = round(df_test['approx_in_show_call']).astype('int')

df_test['쇼호스트_num'] = df_test['쇼호스트명'].apply(lambda x: 0 if x=='없음' else len(x.split(', ')))
df_test['상품중분류_num'] = df_test['상품중분류명'].apply(lambda x: len(set(x)))
df_test['브랜드_num'] = df_test['브랜드명'].apply(lambda x: len(set(x)))
df_test['판매상품개수'] = df_test['상품중분류명'].apply(lambda x: len(x))

df_test['상품판매가_min'] = df_test['상품판매가'].apply(lambda x: min(x))
df_test['상품판매가_max'] = df_test['상품판매가'].apply(lambda x: max(x))
df_test['상품판매가_mean'] = df_test['상품판매가'].apply(lambda x: np.mean(x))

In [17]:
df['인입콜_60분정규화'] = df['인입콜'] / df['방송길이'] * 60
# df['approx_in_show_call_60분정규화'] = df['approx_in_show_call'] / df['방송길이'] * 60

In [18]:
df_test['인입콜_60분정규화'] = df_test['인입콜'] / df_test['방송길이'] * 60
# df_test['approx_in_show_call_60분정규화'] = df_test['approx_in_show_call'] / df_test['방송길이'] * 60

In [19]:
def make_score(df,target,col_name,only_unique):
  score = dict()
  for call, brand_list in zip(df[f'{target}_60분정규화'],df[col_name]):
    for brand in brand_list:
      if brand not in score.keys():
        if only_unique == False:
          score[brand] = [call, list(brand_list).count(brand)]
        else:
          score[brand] = [call, 1]
      else:
        if only_unique == False:
          score[brand][0] += call
          score[brand][1] += list(brand_list).count(brand)
        else:
          score[brand][0] += call
          score[brand][1] += 1
  
  score = pd.DataFrame(score.items())
  score[f'{target}_60분정규화_총합'] = score[1].apply(lambda x: x[0])
  score['방송등장횟수'] = score[1].apply(lambda x: x[1])
  score[f'{target}_60분정규화_평균'] = score[f'{target}_60분정규화_총합'] / score['방송등장횟수']
  score = score.rename(columns={0:col_name})
  score = score.drop(1,axis=1)
  score = score.sort_values(by=[f'{target}_60분정규화_평균','방송등장횟수'],ascending=False).reset_index(drop=True)

  return score

In [20]:
def make_top_col(brand_list,score_df,col_name):
  temp = score_df
  for brand in set(brand_list):
    if brand not in list(score_df[col_name]):
      new = pd.DataFrame([[brand,0,0,score_df[score_df.columns[-1]].mean()]], columns=score_df.columns)
      temp = temp.append(new,ignore_index=True)
    else:
      continue
  
  temp = temp.sort_values(by=[temp.columns[-1],'방송등장횟수'],ascending=False).reset_index(drop=True)
  temp2 = list(temp[temp[col_name].isin(list(set(brand_list)))][col_name])
  return temp2

In [21]:
target = '인입콜' # 'approx_in_show_call'

In [22]:
df2 = df[df[target]!=0]
df2_test = df_test[df_test[target]!=0]

In [23]:
selected_cols = ['PGM코드','상품구분','쇼호스트명','방송길이','기온','상품중분류명','브랜드명','한정표현구분','주말제외공휴일',
                 'MONTH_SIN','MONTH_COS','WEEKDAY_SIN','WEEKDAY_COS','HOUR_START_SIN','HOUR_START_COS','HOUR_END_SIN','HOUR_END_COS','DAY',
                 '쇼호스트_num','상품중분류_num','브랜드_num','한정표현_num','판매상품개수','상품판매가_min','상품판매가_max','상품판매가_mean'] + [target,f'{target}_60분정규화']

x_train = pd.concat([df2[selected_cols],df2_test[selected_cols]]).reset_index(drop=True)

In [24]:
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer(method='box-cox')

from category_encoders.cat_boost import CatBoostEncoder
cat_encoder = CatBoostEncoder()

In [25]:
# Test
mid_cat_score = make_score(x_train,target,'상품중분류명',False)
brand_score = make_score(x_train,target,'브랜드명',False)
limit_score = make_score(x_train,target,'한정표현구분',False)

x_train['top_상품중분류'] = x_train['상품중분류명'].apply(lambda a: make_top_col(a,mid_cat_score,'상품중분류명'))
x_train['상품중분류1'] = x_train['top_상품중분류'].apply(lambda a: a[0])
x_train['상품중분류2'] = x_train['top_상품중분류'].apply(lambda a: a[1] if len(a)>=2 else a[0])
x_train['상품중분류3'] = x_train['top_상품중분류'].apply(lambda a: a[2] if len(a)>=3 else a[0])
x_train['top_브랜드'] = x_train['브랜드명'].apply(lambda a: make_top_col(a,brand_score,'브랜드명'))
x_train['브랜드1'] = x_train['top_브랜드'].apply(lambda a: a[0])
x_train['브랜드2'] = x_train['top_브랜드'].apply(lambda a: a[1] if len(a)>=2 else a[0])
x_train['브랜드3'] = x_train['top_브랜드'].apply(lambda a: a[2] if len(a)>=3 else a[0])
x_train['top_한정표현'] = x_train['한정표현구분'].apply(lambda a: make_top_col(a,limit_score,'한정표현구분'))
x_train['한정표현1'] = x_train['top_한정표현'].apply(lambda a: a[0])
x_train['한정표현2'] = x_train['top_한정표현'].apply(lambda a: a[1] if len(a)>=2 else a[0])
x_train['한정표현3'] = x_train['top_한정표현'].apply(lambda a: a[2] if len(a)>=3 else a[0])

drop_cols = ['상품중분류명','브랜드명','한정표현구분','top_상품중분류','top_브랜드','top_한정표현'] + [target,f'{target}_60분정규화']
y_train = x_train[target]
x_train = x_train.drop(drop_cols,axis=1)

In [26]:
x_train.to_pickle('/content/drive/Shareddrives/현대홈쇼핑/Notebooks/승훈/X_train.pkl')
# y_test.to_pickle('/content/drive/Shareddrives/현대홈쇼핑/Notebooks/승훈/y_test(2201).pkl')

In [27]:
cat_encoder.fit(x_train,y_train)
x_train = cat_encoder.transform(x_train)

y_train = np.reshape(list(y_train),(len(y_train),1))
pt.fit(y_train)
y_train = pt.transform(y_train).reshape(-1)

In [ ]:
model.fit(x_train.values, y_train)

In [ ]:
import joblib

# mid_cat_score.to_pickle('/content/drive/Shareddrives/현대홈쇼핑/Notebooks/승훈/mid_cat_score.pkl')
# brand_score.to_pickle('/content/drive/Shareddrives/현대홈쇼핑/Notebooks/승훈/brand_score.pkl')
# limit_score.to_pickle('/content/drive/Shareddrives/현대홈쇼핑/Notebooks/승훈/limit_score.pkl')

joblib.dump(model, '/content/drive/Shareddrives/현대홈쇼핑/Notebooks/승훈/final_model.pkl')
joblib.dump(cat_encoder, '/content/drive/Shareddrives/현대홈쇼핑/Notebooks/승훈/cat_encoder.pkl')
joblib.dump(pt, '/content/drive/Shareddrives/현대홈쇼핑/Notebooks/승훈/boxcox_transformer.pkl')